Get the training data, and restructure for modeling.

In [4]:
import pandas as pd

data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3.csv")
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['hour'] = data['DateTime'].dt.hour
data['month'] = data['DateTime'].dt.month
data['day'] = data['DateTime'].dt.day
print(data.head(n=15))

# Meal is null, waiting to be filled in.
predData = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3test.csv")
predData['DateTime'] = pd.to_datetime(predData['DateTime'])
predData['hour'] = predData['DateTime'].dt.hour
predData['month'] = predData['DateTime'].dt.month
predData['day'] = predData['DateTime'].dt.day

      id            DateTime  Total  Discounts  ...  meal  hour  month  day
0   9AYC 2019-01-07 08:02:00   3.96       0.00  ...     0     8      1    7
1   6IPU 2019-01-07 08:03:00   3.57       0.00  ...     0     8      1    7
2   IIG5 2019-01-07 08:07:00   2.21       0.00  ...     0     8      1    7
3   RG5T 2019-01-07 08:22:00   2.84       0.00  ...     0     8      1    7
4   9F4F 2019-01-07 08:24:00   4.58       0.00  ...     0     8      1    7
5   6VKI 2019-01-07 08:34:00   3.42       0.00  ...     0     8      1    7
6   JA0W 2019-01-07 08:42:00   1.55       0.00  ...     0     8      1    7
7   69X7 2019-01-07 08:46:00   1.23       0.00  ...     0     8      1    7
8   XE86 2019-01-07 08:48:00   2.21       0.00  ...     0     8      1    7
9   NR24 2019-01-07 08:49:00   3.94       0.00  ...     0     8      1    7
10  7BD6 2019-01-07 08:51:00   4.69       0.00  ...     0     8      1    7
11  IFK7 2019-01-07 08:52:00   1.74       0.00  ...     0     8      1    7
12  83W3 201

### Decision Tree
First model to generate, the most basic.
Accuracy around 87%.

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# For testing
import numpy as np

#x = data.drop(['id', 'DateTime', 'meal'], axis=1) # Drop string, timestamp, and dependent var.
x = data[['Total', 'Discounts', 'hour', 'month', 'day']]
y = data['meal']

# Create model.
model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=15, min_samples_leaf=10)
modelFit = model.fit(x,y)

# Test our model using the testing data
predData = predData[['Total', 'Discounts', 'hour', 'month', 'day']]
pred = modelFit.predict(predData)

dataTest = pd.read_csv("tests/testData.csv")['meal']

def tjurr(truth, pred):
    truth = list(dataTest)
    predTjurr = list(pred)
    y1 = np.mean([y for x, y in enumerate(predTjurr) if truth[x]==1])
    y2 = np.mean([y for x, y in enumerate(predTjurr) if truth[x]==0])
    return y1-y2

tjurr(dataTest, pred)

np.float64(0.31137041465660603)

### Random Forest
Accuracy is around 82%.

In [36]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#x = data.drop(['id', 'DateTime', 'meal'], axis=1) # Drop string, timestamp, and dependent var.
x = data[['Total', 'Discounts', 'hour', 'month', 'day']]
y = data['meal']

# Create model.
model = RF(n_estimators=200, n_jobs=-1, max_depth=15, min_samples_leaf=2, min_samples_split=5)
modelFit = model.fit(x,y)

# Test our model using the testing data
predData = predData[['Total', 'Discounts', 'hour', 'month', 'day']]
pred = modelFit.predict(predData)

dataTest = pd.read_csv("tests/testData.csv")['meal']

def tjurr(truth, pred):
    truth = list(dataTest)
    pred = list(pred)
    y1 = np.mean([y for x, y in enumerate(pred) if truth[x]==1])
    y2 = np.mean([y for x, y in enumerate(pred) if truth[x]==0])
    return y1-y2

tjurr(dataTest, pred)

np.float64(0.12621119036820186)

### XGBoost


In [35]:
from xgboost import XGBClassifier
# For testing
import numpy as np

#x = data.drop(['id', 'DateTime', 'meal'], axis=1) # Drop string, timestamp, and dependent var.
x = data[['Total', 'Discounts', 'hour', 'month', 'day']]
y = data['meal']

# Create model
model = XGBClassifier(
    n_estimators=150, 
    max_depth=15,
    learning_rate=0.5,
    objective='binary:logistic'
)

modelFit = model.fit(x,y)

# Test our model using the testing data
predData = predData[['Total', 'Discounts', 'hour', 'month', 'day']]
pred = modelFit.predict(predData)

dataTest = pd.read_csv("tests/testData.csv")['meal']

def tjurr(truth, pred):
    truth = list(dataTest)
    pred = list(pred)
    y1 = np.mean([y for x, y in enumerate(pred) if truth[x]==1])
    y2 = np.mean([y for x, y in enumerate(pred) if truth[x]==0])
    return y1-y2

tjurr(dataTest, pred)
# np.float64(0.09872318201184732)

np.float64(0.21119036820187193)

#### Cross Validation
For XGBoost model.

In [53]:
from sklearn.model_selection import KFold

# If we have imported data and created x, y already:
kf = KFold(n_splits=10) # 10 "Folds"

models = [] # We will store our models here

for train, test in kf.split(x): # Iterate over folds
    model = XGBClassifier(
            n_estimators=50, 
            max_depth=10,
            learning_rate=0.5
                ).fit(x.values[train], y.values[train]) # Fit model
    accuracy = accuracy_score(y.values[test],    # Store accuracy
        model.predict(x.values[test]))
    print("Accuracy: ", accuracy)            # Print results
    models.append([model, accuracy])      # Store it all

print("Mean Model Accuracy: ", np.mean([model[1] for model in models]))
print("Model Accuracy Standard Deviation: ", np.std([model[1] for model in models]))

Accuracy:  0.8901209677419355
Accuracy:  0.8649193548387096
Accuracy:  0.8850806451612904
Accuracy:  0.8719758064516129
Accuracy:  0.8719758064516129
Accuracy:  0.8679435483870968
Accuracy:  0.8699596774193549
Accuracy:  0.8790322580645161
Accuracy:  0.8940464177598385
Accuracy:  0.8890010090817356
Mean Model Accuracy:  0.8784055491357702
Model Accuracy Standard Deviation:  0.009921935669179163
